In [174]:
import numpy as np

# Setting seed, just so you get similar results in case you want to use this code.
np.random.seed(42)

#### To make these Random paths, we will use Geometric Brownian Motion, our simple assumption here is that the logarithm of stock prices follows a Geometric Brownian Motion.

![](https://jdsupra-html-images.s3-us-west-1.amazonaws.com/36ac8b9f-d89d-4e5b-a62c-33e1f441d118-Geometric_Brownian_Motion_graphic_Opportune_LLP.png)



#### S<sub>t</sub> is the stock price at time t, S<sub>0</sub> is price at time t = 0.

#### (μ - σ<sup>2</sup>/2)t is also known as **drift**

#### ε is a random variable that follows Standard Normal Distribution ~ N(0,1)

In [175]:
# Initialize some parameters

stockPrice = 1000 # The actual spot/underlying price of the asset
timeToExpiration = 1/12  # 1 month 
riskFreeRate = 0.04 # Risk Free Interest Rate
volatility = 0.2 # Annual Risk/Standard Deviation/Volatility of the underlying stock/asset
strikePrice = 1020


numSimulations = 5000 # Number of Monte Carlo Simulations

numDays = 21 # Roughly 21 Trading days in a month on an average



In [176]:
# Since we are doing this for daily prices, we need to normalize t

t = timeToExpiration / numDays

In [177]:
# Let's define drift

drift = (riskFreeRate - (volatility**2)/2) * t

# Let's assign σ * √t to another variable to make things easier

a = volatility * np.sqrt(t)

# Let's define ε

epsilon = np.random.normal(0,1,(numSimulations,numDays))

epsilon.shape

(5000, 21)

In [178]:
priceMatrix = np.zeros((numSimulations,numDays))

print(priceMatrix.shape)

priceMatrix

(5000, 21)


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [179]:
# Assign the initial stock price value in the first column(t=0)

priceMatrix[:,0] += stockPrice

# Now let's start filling other column values using GBM(Geometric Brownian motion)
for i in range(1,numDays):
    priceMatrix[:,i] += priceMatrix[:,i-1] * np.exp(drift + a * epsilon[:,i])

In [180]:
priceMatrix

array([[1000.        ,  998.33877998, 1006.59854738, ...,  970.25217711,
         953.21647127,  971.05861197],
       [1000.        , 1000.9305732 ,  983.20205134, ...,  921.99840725,
         930.69038995,  932.77598296],
       [1000.        ,  996.29270414,  977.98353351, ...,  978.88646986,
         976.6769597 ,  963.23441791],
       ...,
       [1000.        ,  991.28232493,  988.97715768, ...,  913.32364137,
         902.40042845,  898.23443517],
       [1000.        , 1006.87357652, 1012.74313696, ..., 1014.38166864,
        1013.33183478, 1004.17524221],
       [1000.        ,  985.48719324,  992.24428683, ..., 1040.61533222,
        1036.62677061, 1043.64366385]])

#### Let's calculate the payoffs now

In [181]:
# Payoff for call = Last value in priceMatrix(spot value at expiration) - strikePrice

callPayoffMatrix = priceMatrix[:,-1] - strikePrice

# Eliminate Negative Payoffs

for i in range(len(callPayoffMatrix)):
    if callPayoffMatrix[i] < 0:
        callPayoffMatrix[i] = 0
    
# Payoff for Put = strikePrice - Last value in priceMatrix(spot value at expiration)

putPayoffMatrix = strikePrice - priceMatrix[:,-1] 

# Eliminate Negative Payoffs

for i in range(len(putPayoffMatrix)):
    if putPayoffMatrix[i] < 0:
        putPayoffMatrix[i] = 0
    

In [182]:
callPayoff = callPayoffMatrix.mean()
putPayoff = putPayoffMatrix.mean()

In [183]:
callPayoff, putPayoff

(15.682364946825674, 32.36537917718196)

In [184]:
# Continous discounting for today's price

call = callPayoff * np.exp(-riskFreeRate * timeToExpiration)
put = putPayoff * np.exp(-riskFreeRate * timeToExpiration)

In [185]:
call, put


(15.63017742452852, 32.25767418796686)

* #### The Call Option(Out of the money - Only has time value) is priced at 15.63 for a strike price of 1020 when the underlying stock price is 1000.

* #### The Put Option(In the Money - has both Time value and intrinsic value) is priced at 32.25.

* #### Next steps - Compare the prices with Black Scholes Options Pricing Model (you will find that the prices are quite similar)

